In [40]:
import tensorflow as tf
# tf.compat.v1.disable_eager_execution()
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

learning_rate = 1e-3
keep_prob_rate = 0.7 # 
max_epoch = 3
batch_size = 100
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
    correct_prediction = tf.compat.v1.equal(tf.compat.v1.argmax(y_pre,1), tf.compat.v1.argmax(v_ys,1))
    accuracy = tf.compat.v1.reduce_mean(tf.compat.v1.cast(correct_prediction, tf.compat.v1.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
    return result

def weight_variable(shape):
    initial = tf.compat.v1.random.truncated_normal(shape, stddev=0.1, mean=0.0)
    return tf.compat.v1.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.compat.v1.Variable(initial)

def conv2d(x, W):
    # 每一维度  滑动步长全部是 1， padding 方式 选择 same
    # 提示 使用函数  tf.nn.conv2d
    out = tf.compat.v1.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
    return out

def max_pool_2x2(x):
    # 滑动步长 是 2步; 池化窗口的尺度 高和宽度都是2; padding 方式 请选择 same
    # 提示 使用函数  tf.nn.max_pool
    out = tf.compat.v1.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    return out

# define placeholder for inputs to network
xs = tf.compat.v1.placeholder(tf.compat.v1.float32, [None, 784])/255.
ys = tf.compat.v1.placeholder(tf.compat.v1.float32, [None, 10])
keep_prob = tf.compat.v1.placeholder(tf.compat.v1.float32)
x_image = tf.compat.v1.reshape(xs, [-1, 28, 28, 1])

#  卷积层 1
## conv1 layer ##

W_conv1 = weight_variable([7, 7, 1, 32])  # patch 7x7, in size 1, out size 32
b_conv1 = bias_variable([32])                     
h_conv1 = tf.compat.v1.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  # 卷积  自己选择 选择激活函数
h_pool1 = max_pool_2x2(h_conv1)           # 池化               

# 卷积层 2
W_conv2 = weight_variable([5, 5, 32, 64])  # patch 5x5, in size 32, out size 64
b_conv2 = bias_variable([64])                     
h_conv2 = tf.compat.v1.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)    # 卷积  自己选择 选择激活函数
h_pool2 = max_pool_2x2(h_conv2)             # 池化

#  全连接层 1
## fc1 layer ##
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.compat.v1.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.compat.v1.nn.relu(tf.compat.v1.matmul(h_pool2_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.compat.v1.nn.dropout(h_fc1, keep_prob)

# 全连接层 2
## fc2 layer ##
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
# logits用于计算损失，而经过softmax层的输出则是预测结果
logits = tf.compat.v1.matmul(h_fc1_drop, W_fc2) + b_fc2  
prediction = tf.compat.v1.nn.softmax(logits)


# 交叉熵函数
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.compat.v1.log(prediction),axis=1))
cross_entropy = tf.compat.v1.nn.softmax_cross_entropy_with_logits(labels=ys, logits=logits)
train_step = tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

with tf.compat.v1.Session() as sess:
    init = tf.compat.v1.global_variables_initializer()
    sess.run(init)
    
    for i in range(max_epoch):
        total_batches = mnist.train.num_examples // batch_size
        for j in range(total_batches):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys, keep_prob: keep_prob_rate})
        
        # 在每个epoch结束时评估模型准确率
        test_accuracy = compute_accuracy(mnist.test.images[:1000], mnist.test.labels[:1000])
        print(f'step {i}, test accuracy {test_accuracy}')

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
step 0, test accuracy 0.9860000014305115
step 1, test accuracy 0.9890000224113464
step 2, test accuracy 0.9890000224113464
